In [36]:
import cv2
import numpy as np

class Contour:
    def __init__(self, coordinates, color):
        self.coordinates = coordinates
        self.color = color

# Define a function that returns the x, y coordinates of the contour
def get_contour_center(contour):
    contour_np = np.array(contour)
    rect = cv2.minAreaRect(contour_np)
    center = rect[0]
    return center

# Load the image
img = cv2.imread("IMG_4742.jpg")

# Convert the image to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define the range of color you want to detect

color_lower = (20, 100, 100)
color_upper = (40, 255, 255)

colors = [('yellow', (20, 100, 100), (40, 255, 255)),
    ('red', (0, 100, 100), (10, 255, 255)),
    ('blue', (100, 50, 50), (120, 255, 255)),
    ('green', (50, 50, 50), (70, 255, 255)),
    ('orange', (5, 50, 50), (15, 255, 255)),
    ('white', (0, 0, 200), (180, 30, 255))
    ]

color_map = {"red": (0, 0, 255), "green": (0, 255, 0), "blue": (255, 0, 0), "yellow": (0, 255, 255), "white": (255,255,255), "orange": (0, 165, 255)}

contoursList = []

for color, color_lower, color_upper in colors:
    # Create a binary mask where the color is within the range
    mask = cv2.inRange(hsv, color_lower, color_upper)

    # Perform morphological operations and draw squares onto the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours on the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        # Approximate the contour with a polygon
        epsilon = 0.02*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        area = cv2.contourArea(cnt)
        rect = cv2.minAreaRect(approx)
        angle = rect[-1]
        # check if the approximated contour has 4 sides
        if len(approx) == 4 and area > 100000 and (abs(angle) < 10 or abs(angle) > 80):
            # Draw the contour on the original image
            contoursList.append(Contour(cnt, color))

# Sort the contours_list
contoursList = sorted(contoursList, key=lambda c: (cv2.boundingRect(c.coordinates)[1], cv2.boundingRect(c.coordinates)[0]))

counter = 0
for contour in contoursList:
    counter += 1
    color = color_map[contour.color]
    # Draw the contour on the image
    cv2.drawContours(img,[contour.coordinates], 0, color, 2)
    cX = contour.coordinates[0][0][0]
    cY = contour.coordinates[0][0][1]
    print (counter)
    cv2.putText(img, 'piece #' + str(counter), (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 8)




# Display the original image with the squares highlighted
cv2.imshow("Squares", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(33)

1
2
3
4
5
6
7
8
9


-1